<h1 align="center">Hacettepe University<br>Department of Computer Science</h1> 
<h2 align="center">BBM 418 - Computer Vision Laboratory<br>2022 Spring</h2>
<h3 align="center">Assignment 4</h3>
<h3 align="left">Name    :    Murat Çelik</h3>
<h3 align="left">ID      :    21827263</h3>
<h4 align="right">Due on May 25, 2022</h4>

## Table of Contents
- [1 - PART 2: Object Tracking use YOLOv3 and Mean-Shift](#1)
    - [1.1 - Required Libraries](#1.1)
    - [1.2 - Detect and Save data with YOLOv3](#1.2)
    - [1.3 - Filter the Main Person in Videos](#1.3)
    - [1.4 - Video Creation with Filtered Data](#1.4)
    - [1.5 - Video Creation with Filtered Data on Mean-Shift Algorithm](#1.5)
      - [1.5.1 - Mean-Shift Algorithm](#1.5.1)
    - [1.6 - Experimental Results](#1.6)
    - [1.7 - Conclusion](#1.7)
    - [1.8 - References](#1.8)

# PART 2: Object Tracking use YOLOv3 and Mean-Shift <a name='1'></a>
- In this section, it is to detect the people in the video and estimate their location. This is called tracking by detection. For this, the YOLOv3 algorithm and the Mean Shift algorithm are used.

- The YOLOv3 algorithm is tested on a few videos. And the data is obtained. Then, the Mean Shift algorithm is used in addition to the algorithm in the parts that YOLOv3 cannot detect and are missing. All these results are analyzed.

## 1.1 Required Libraries <a name='1.1'></a>


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
try :
  os.chdir("drive/My Drive")
  os.chdir("BBM418/Assignment 4")
  os.chdir("Part 2")
except: 
  print("Already set")

In [ ]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow
import matplotlib.pyplot as plt
import pandas as pd

## 1.2 Detect and Save data with YOLOv3 <a name='1.2'></a>

YOLOv3 (You Only Look Once, Version 3) is a real-time object detection algorithm that identifies specific objects in videos, live feeds, or images. YOLO uses features learned by a deep convolutional neural network to detect an object. Versions 1-3 of YOLO were created by Joseph Redmon and Ali Farhadi.

In [ ]:
def yolo_v3(img, net, classes, output_layers, height_origi, width_origi, colors = (0,255,0)):
    # prepare model 
    height, width, channels = img.shape
    blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    outs = net.forward(output_layers)

    class_ids = []
    confidences = []
    boxes = []
    
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:
                # Object detected
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)
                # Rectangle coordinates
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)
                boxes.append([x, y, w, h]) # save boxes
                confidences.append(float(confidence))
                class_ids.append(class_id)
    # apply non maximum suppreison on predictions
    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
    font = cv2.FONT_HERSHEY_PLAIN
    boxes_file = list()
    for i in range(len(boxes)):
        if i in indexes:
            x, y, w, h = boxes[i]
            if class_ids[i] == 0:
              label = str(classes[class_ids[i]])
              x1,y1,w1,h1 = int(x/416*width_origi), int(y/416*height_origi), int(w/416*width_origi), int(h/416*height_origi)
              boxes_file.append([x1,y1,w1,h1]) # save data in original scale
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    return img, boxes_file

In [ ]:
def one_video(images, video_name, data_df, net, classes, output_layers):
  image_path = images[0]
  frame = cv2.imread(image_path)
  height, width, channels = frame.shape
  
  for count,image in enumerate(images,start=1):
      image_map = dict() # save all data as a dataFrame
      image_map["Video Name"] = video_name
      image_map["Image Name"] = image.split("/")[-1]
      image_map["Frame"] = count

      frame = cv2.imread(image)
      height, width, channels = frame.shape
      frame = cv2.resize(frame, (416, 416))
      # run yolo function and get predictions
      frame,boxes = yolo_v3(frame, net, classes, output_layers, height, width)

      for box in boxes:
        image_map["X"] = box[0]
        image_map["Y"] = box[1]
        image_map["W"] = box[2]
        image_map["H"] = box[3]
        data_df = data_df.append(image_map, ignore_index=True)
      frame = cv2.resize(frame, (width, height))

      if (cv2.waitKey(1) & 0xFF) == ord('q'): # Hit `q` to exit
          break
  cv2.destroyAllWindows()
  return data_df

The paths of the images to be processed are recorded in the list.

In [ ]:
DATA_PATH = 'content/vot2017/'
person_track_subfolder = ["gymnastics1", "iceskater1", "pedestrian1"]
data_path = dict()
for subfolder in person_track_subfolder:
  path = DATA_PATH + subfolder + "/color"
  path_image_list = list()
  for insubfolder in sorted(os.listdir(path)):
    path_image_list.append(path + "/" + insubfolder)
  
  data_path[subfolder] = path_image_list

The YOLOv3 model is created in the code below. All people in each frame are detected. These detected data are saved in the file for each label with the name "< label >.csv". All data can be found in the folder.

In [ ]:
for label in person_track_subfolder:
  net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg") # create model with weights
  classes = ["person"]
  layer_names = net.getLayerNames()
  output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]
  df = pd.DataFrame(columns=["Video Name", "Image Name", "Frame", "X", "Y", "W", "H"])
  df = one_video(data_path[label], label, df, net, classes, output_layers)
  df.to_csv(label + "/" + label + ".csv")

## 1.3 Filter the Main Person in Videos <a name='1.3'></a>
The YOLOv3 algorithm detects every person in the videos. In this study, analyzes are made for a single person in the video. For this reason, YOLOv3's detections are filtered until only one box remains for each frame. This filtering process is done by calculating the IoU value of the ground truth data and the data of the YOLOv3 algorithm. If the obtained IoU score is above 40%, it is kept, otherwise it is deleted. Thus, the YOLOv3 algorithm produces results only for the object studied in the video.

In [ ]:
def IoU(y_test, y_pred, threshold):
  x1,y1,w_test,h_test = y_test
  x2,y2 = x1 + w_test, y1 + h_test
  xx1, yy1, w_pred, h_pred = y_pred
  xx2,yy2 = xx1 + w_pred, yy1 + h_pred
  if intersect([x1,y1,x2,y2], [xx1,yy1,xx2,yy2]):
    return False, 0
  x_inter1 = max(x1,xx1)
  y_inter1 = max(y1,yy1)
  x_inter2 = min(x2,xx2)
  y_inter2 = min(y2,yy2)
  width_inter = abs(x_inter2 - x_inter1)
  height_inter = abs(y_inter2 - y_inter1)
  area_inter = height_inter * width_inter
  area_1 = abs(x2-x1) * abs(y2-y1)
  area_2 = abs(xx2-xx1) * abs(yy2-yy1)
  area_union = area_1 + area_2 - area_inter
  iou = area_inter / area_union
  return iou > threshold, iou
def intersect(y_test, y_pred):
  x1,y1,x2,y2 = y_test
  xx1,yy1,xx2,yy2 = y_pred
  if x2 < xx1 or xx2 < x1:
    return True
  if y2 < yy1 or yy2 < y1:
    return True

In [ ]:
def filter_yolo(file_name):
  file = "content/vot2017/"+file_name+"/groundtruth.txt"
  f = open(file,"r")
  frame_gt = list()
  for x in f:
    gt_one_frame = list(map(int,list(map(float,x.split(",")))))
    x = gt_one_frame[0] 
    y = gt_one_frame[1] 
    w = abs(gt_one_frame[0] - gt_one_frame[4])
    h = abs(gt_one_frame[1] - gt_one_frame[5])
    frame_gt.append([x,y,w,h])
  f.close()
  df = pd.read_csv(file_name + "/" + file_name + ".csv")
  df_filter = pd.DataFrame(columns = ["Image Name", "X", "Y", "W" ,"H"])
  for count,frame in enumerate(sorted(df["Image Name"].value_counts().keys())):
    df_tmp = df[df["Image Name"] == frame]
    y_test = frame_gt[count]
    for y in range(len(df_tmp)):
      tmp_row = df_tmp.iloc[y]
      x,y,w,h = tmp_row["X"], tmp_row["Y"], tmp_row["W"], tmp_row["H"],
      y_pred_tmp = [x,y,w,h]
      y_pred = y_pred_tmp.copy()

      control, iou = IoU(y_test.copy(), y_pred, 0.4)
      if control:
        df_filter = df_filter.append({"Image Name" : frame, "X": x, "Y":y, "W":w, "H" : h}, ignore_index=True)
  df_filter.to_csv(label + "/" + label +"_filter.csv")
  return df_filter, frame_gt

 ## 1.4 Video Creation with Filtered Data <a name='1.4'></a>

In this section, there are two different data for each frame. One of these data is ground truth data. The other is the data that YOLOv3 predicts. It is known that ground truth data contains one data for each frame. Since YOLOv3's predictions have deficiencies and missed predictions, it cannot be said to contain one data for each frame.

When creating the video, 2 colors are used for the boxes. *The **blue** ones of these boxes* show ground truth data. *The  **green** one of these boxes* shows the data of the YOLOv3 algorithm.

In [ ]:
def write_video(data_path, frame_gt,video_name, df):
  colors = (0,255,0)
  images = data_path[video_name]
  image_path = images[0]
  frame = cv2.imread(image_path)
  height, width, channels = frame.shape
  fourcc = cv2.VideoWriter_fourcc(*'mp4v') 
  writer = cv2.VideoWriter(video_name + "/" + video_name +"without_meanshift.mp4", fourcc, 24, (width, height))
  for count,image in enumerate(images,start=1):
      frame = cv2.imread(image)
      height, width, channels = frame.shape
      # frame = cv2.resize(frame, (416, 416))
      df_filter_image = df[df["Image Name"] == image.split("/")[-1]]
      for boxes in range(len(df_filter_image)):
        box = df_filter_image.iloc[boxes]
        x,y,w,h = box["X"], box["Y"], box["W"], box["H"]
        cv2.rectangle(frame, (x, y), (x + w, y + h), colors, 1)
      y_test = frame_gt[count-1]
      x,y,w,h = y_test[0], y_test[1],  y_test[2], y_test[3]
      cv2.rectangle(frame, (x, y), (x + w, y + h), (255,0,0), 1)
      # frame = cv2.resize(frame, (width, height))
      cv2_imshow(frame)
      writer.write(frame) # Write out frame to video

      #v2_imshow(frame)
      if (cv2.waitKey(1) & 0xFF) == ord('q'): # Hit `q` to exit
          break

  # Release everything if job is finished
  writer.release()
  cv2.destroyAllWindows()

In [ ]:
for label in person_track_subfolder:
  df_filter, frame_gt = filter_yolo(label)
  df_filter.to_csv(label + "/" + label+ "_filter.csv")
  write_video(data_path, frame_gt, label , df_filter)

 ## 1.5 Video Creation with Filtered Data on Mean Shift Algorithm <a name='1.5'></a>

In this section, the Mean Shift Algorithm is used. With this algorithm, the previous prediction data is analyzed for the frames where YOLOv3 is missing, and predictions are made. Thus, there is a chance to follow the tracked object for each frame.

 When creating the video, 3 colors are used for the boxes. *The **blue** ones of these boxes* show ground truth data. T*he **green** one of these boxes* shows the data of the YOLOv3 algorithm. *The **red** one of these boxes* is the data of the **Mean-Shift algorithm**.

 ### 1.5.1 Mean-Shift Algorithm <a name='1.5.1'></a>
 The Mean Shift algorithm has a basic logic for object tracking. It makes inferences from previous predictions for unpredicted frames. The position of the object in the previous picture is given. High concentration of data points of that object are found. For the next frame, similar data points are scanned around that object and a new estimate is made.

In [ ]:
def write_video_with_meanShift(data_path, frame_gt,video_name, df):
  # Store data as dataframe
  df_filter_meanShift = pd.DataFrame(columns = ["Image Name", "X", "Y", "W" ,"H"])
  images = data_path[video_name] 
  image_path = images[0]
  frame = cv2.imread(image_path)
  height, width, channels = frame.shape
  # Video writer algorithm
  fourcc = cv2.VideoWriter_fourcc(*'mp4v') 
  writer = cv2.VideoWriter(video_name + "/" + video_name +"_meanshift.mp4", fourcc, 24, (width, height))
  x_last,y_last,w_last, h_last = 0,0,0,0 # The most recent estimate.
  previous_not_meanshift_control = True # Mean-Shift or YOLOV3 made the previous prediction.
  x,y,w,h = 0,0,0,0
  for count,image in enumerate(images,start=1):
      frame = cv2.imread(image)
      height, width, channels = frame.shape
      # frame = cv2.resize(frame, (416, 416))
      frame_name = image.split("/")[-1]
      df_filter_image = df[df["Image Name"] == frame_name]
      for boxes in range(len(df_filter_image)):
        box = df_filter_image.iloc[boxes]
        x,y,w,h = box["X"], box["Y"], box["W"], box["H"]
        df_filter_meanShift = df_filter_meanShift.append({"Image Name" : frame_name, "X": x, "Y":y, "W":w, "H" : h}, ignore_index=True)
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0,255,0), 1)
      if x == 0 and y==0 and w==0 and h==0:
        continue
      # The part where the Mean-Shift Algorithm is used.
      if len(df_filter_image) == 0:
        x, y, w, h = x_last,y_last,w_last, h_last
        track_window = (x, y, w, h)
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        dst = cv2.calcBackProject([hsv],[0],roi_hist,[0,180],1)
        # apply meanshift to get the new location
        ret, track_window = cv2.meanShift(dst, track_window, term_crit)
        # Draw it on image
        x,y,w,h = track_window
        x_last,y_last,w_last, h_last = track_window
        df_filter_meanShift = df_filter_meanShift.append({"Image Name" : frame_name, "X": x, "Y":y, "W":w, "H" : h}, ignore_index=True)
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0,0,255), 1)
        previous_not_meanshift_control = False
      # Holds the last valid predictions for the Mean Shift algorithm.
      if(previous_not_meanshift_control):
        roi = frame[y:y+h, x:x+w]
        hsv_roi =  cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
        mask = cv2.inRange(hsv_roi, np.array((0., 60.,32.)), np.array((180.,255.,255.)))
        roi_hist = cv2.calcHist([hsv_roi],[0],mask,[180],[0,180])
        cv2.normalize(roi_hist,roi_hist,0,255,cv2.NORM_MINMAX)
        term_crit = ( cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 1 )
      # Draws ground truth data.
      y_test = frame_gt[count-1]
      x_last,y_last,w_last, h_last = x,y,w,h
      x,y,w,h = y_test[0], y_test[1],  y_test[2], y_test[3]
      cv2.rectangle(frame, (x, y), (x + w, y + h), (255,0,0), 1)
      # frame = cv2.resize(frame, (width, height))
      cv2_imshow(frame)
      writer.write(frame) # Write out frame to video
      #v2_imshow(frame)
      if (cv2.waitKey(1) & 0xFF) == ord('q'): # Hit `q` to exit
          break
  writer.release()
  cv2.destroyAllWindows()
  return df_filter_meanShift

In [ ]:
for label in ["pedestrian1"]:
  file = "content/vot2017/"+label+"/groundtruth.txt"
  f = open(file,"r")
  frame_gt = list()
  for x in f:
    gt_one_frame = list(map(int,list(map(float,x.split(",")))))
    x = gt_one_frame[0] 
    y = gt_one_frame[1] 
    w = abs(gt_one_frame[0] - gt_one_frame[4])
    h = abs(gt_one_frame[1] - gt_one_frame[5])
    frame_gt.append([x,y,w,h])
  f.close()
  df_filter = pd.read_csv(label + "/" + label+ "_filter.csv")
  df_filter_meanShift = write_video_with_meanShift(data_path, frame_gt, label, df_filter)
  df_filter_meanShift.to_csv(label + "/" + label+ "_filter_withMeanShift.csv")


Output hidden; open in https://colab.research.google.com to view.

 ## 1.6 Experimental Results <a name='1.6'></a>
We have 2 types of estimation data. One of them contains only YOLOv3's predictions. The other includes predictions from YOLOv3 + Mean-Shift. These data are separately subjected to ground truth data and IoU (Intersection over Union) calculation. A score is kept for each frame. These scores are saved as files in the drive folder. The score of that frame is given as 0 for data with no conflicts or no predictions. All scores are summed and the last stage is divided by the number of frames.


In [ ]:
def IoU(y_test, y_pred):
  x1,y1,w_test,h_test = y_test
  x2,y2 = x1 + w_test, y1 + h_test
  xx1, yy1, w_pred, h_pred = y_pred
  xx2,yy2 = xx1 + w_pred, yy1 + h_pred
  if intersect([x1,y1,x2,y2], [xx1,yy1,xx2,yy2]):
    return 0
  x_inter1 = max(x1,xx1)
  y_inter1 = max(y1,yy1)
  x_inter2 = min(x2,xx2)
  y_inter2 = min(y2,yy2)
  width_inter = abs(x_inter2 - x_inter1)
  height_inter = abs(y_inter2 - y_inter1)
  area_inter = height_inter * width_inter
  area_1 = abs(x2-x1) * abs(y2-y1)
  area_2 = abs(xx2-xx1) * abs(yy2-yy1)
  area_union = area_1 + area_2 - area_inter
  iou = area_inter / area_union
  return iou
def intersect(y_test, y_pred):
  x1,y1,x2,y2 = y_test
  xx1,yy1,xx2,yy2 = y_pred
  if x2 < xx1 or xx2 < x1:
    return True
  if y2 < yy1 or yy2 < y1:
    return True

In [ ]:
def iou_calculate(label, meanshift):
  file = "content/vot2017/"+label+"/groundtruth.txt"
  f = open(file,"r")
  frame_gt = list()
  for x in f:
    gt_one_frame = list(map(int,list(map(float,x.split(",")))))
    x = gt_one_frame[0] 
    y = gt_one_frame[1] 
    w = abs(gt_one_frame[0] - gt_one_frame[4])
    h = abs(gt_one_frame[1] - gt_one_frame[5])
    frame_gt.append([x,y,w,h])
  f.close()
  if meanshift:
    df_filter = pd.read_csv(label + "/" + label+ "_filter_withMeanShift.csv")
  else:
    df_filter = pd.read_csv(label + "/" + label+ "_filter.csv")
  iou_list = dict()
  for count in range(len(frame_gt)):
    frame_num = count + 1
    tmp = len(str(frame_num))
    frame_num_name = str(frame_num)
    for _ in range(tmp,8):
      frame_num_name = "0" + frame_num_name 
    frame_num_name = frame_num_name + ".jpg"
    df_tmp = df_filter[df_filter["Image Name"] == frame_num_name]
    if len(df_tmp) == 0:
      iou_list[frame_num_name] = 0
      continue
    total_iou = 0
    for index in range(len(df_tmp)):
      x,y,w,h = df_tmp.iloc[index]["X"], df_tmp.iloc[index]["Y"], df_tmp.iloc[index]["W"], df_tmp.iloc[index]["H"]
      iou = IoU(frame_gt[count], [x,y,w,h])
      total_iou += iou
    iou_list[frame_num_name] = total_iou / len(df_tmp)
  iou_series = pd.Series(iou_list)
  if meanshift:
    iou_series.to_csv(label + "/" + label + "_iou_meanshift.csv")
  else:
    iou_series.to_csv(label + "/" + label + "_iou_without_meanshift.csv")

In [ ]:
for label in person_track_subfolder[::-1]:
  iou_calculate(label, True)
  iou_calculate(label, False)

In [ ]:
person_track_subfolder = ["gymnastics1", "iceskater1", "pedestrian1"]
df_result = pd.DataFrame(index=["Mean Shift", "Non Mean Shift"])
for label in person_track_subfolder[::-1]:
    iou_series_mean = pd.read_csv(label + "/" + label + "_iou_meanshift.csv")
    score_mean = iou_series_mean["0"].sum() / len(iou_series_mean)

    iou_series_notmean = pd.read_csv(label + "/" + label + "_iou_without_meanshift.csv")
    score_notmean = iou_series_notmean["0"].sum() / len(iou_series_notmean)

    df_result[label] = [score_mean, score_notmean]

When we look at the scores, we can see that the Mean-Shift algorithm improves the results. It is predicted that the score may increase by making predictions on data sets that have never been predicted. It can be said that the algorithm makes good predictions at points of 2-3 frames, but the size of the error increases as the frame to be estimated increases.

Except this:

- It is observed that the ***best score*** is taken from the "***pedestrian1***" dataset. The reason why this score is the best can be said to be the main reason that the YOLOv3 algorithm makes more predictions in the data. Easy to distinguish the human object inside from the background, not including any zoom-in, zoom-out, convenience has been observed here.

- It is observed that the ***average score*** is taken from the "***gymnastics1***" dataset. It can be said that the main reason for this score is the underestimation of the YOLOv3 algorithm in some parts of the data. There are various reasons for this situation. Difficulties have been observed here, as the human object in it cannot be easily distinguished from the background, and it contains zoom-in and zoom-out. There are also incomplete estimations due to the movements of the object in the data. The Mean-Shift algorithm contains incorrect predictions at these points.

- It is observed that the ***worst score*** is taken from the "***iceskater1***" dataset. It can be said that the main reason for this score is the underestimation of the YOLOv3 algorithm in most parts of the data. The second main reason is that movements such as the rotation of the object in the video, shrinking and jumping cause difficulties in estimation. Difficulties have been observed here, as the human object in it cannot be easily distinguished from the background, it contains too much zoom-in and zoom-out. There have been points where the Mean-Shift algorithm has been overused, and these points contain erroneous predictions.

In [ ]:
#df_result.to_csv("accuracy.csv")
df_result = pd.read_csv("accuracy.csv")
df_result

,pedestrian1,iceskater1,gymnastics1
Mean Shift,0.589540,0.351710,0.399947
Non Mean Shift,0.583418,0.291152,0.317797


  ## 1.7 Conclusion <a name='1.7'></a>

In this part, we tested the object tracking algorithm. Videos were processed using YOLOv3 and Mean-Shift algorithms. All data is saved in the drive folder. Information was obtained about the difficulties of object tracking and the solution of these difficulties. Pytorch and Opencv libraries were used and experience was gained in computer vision.

## 1.8 References <a name='1.8'></a>

[Mean-Shift Algorithm](https://docs.opencv.org/3.4/d7/d00/tutorial_meanshift.html)

[YOLO for Object Detection, Architecture Explained!](https://medium.com/analytics-vidhya/understanding-yolo-and-implementing-yolov3-for-object-detection-5f1f748cc63a)

[DataSet](https://www.votchallenge.net/vot2017/dataset.html)

[YOLOv3: Real-Time Object Detection Algorithm (What’s New?)](https://viso.ai/deep-learning/yolov3-overview/)
